# 模型全参数微调，sft, 因为训练的是causalLM, 因此不用划分数据集

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

d:\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("./data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [5]:
print(ds[0])

{'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。', 'input': '', 'instruction': '保持健康的三个提示。'}


## Step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/models--Langboat--bloom-389m-zh")
tokenizer

BloomTokenizerFast(name_or_path='D:/pretrained_model/models--Langboat--bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [18]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [19]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Map: 100%|██████████| 26858/26858 [00:11<00:00, 2245.00 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [21]:
print(tokenizer.decode(tokenized_ds[1]['input_ids']))

Human: 解释为什么以下分数等同于1/4
输入：4/16

Assistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>


In [23]:
# tokenizer.decode(tokenized_ds[1]['labels']) # 不行的有负号
print(tokenizer.decode(list(filter(lambda x: x!=-100, tokenized_ds[1]['labels']))))

4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>


## Step4 创建模型

In [25]:
model = AutoModelForCausalLM.from_pretrained("D:/pretrained_model/models--Langboat--bloom-389m-zh")

d:\anaconda3\envs\py39\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="c

## Step5 配置训练参数

In [26]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [27]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

## Step7 模型训练

In [28]:
trainer.train()

d:\anaconda3\envs\py39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 10/1678 [00:13<35:34,  1.28s/it]

{'loss': 3.279, 'learning_rate': 4.9702026221692494e-05, 'epoch': 0.01}


  1%|          | 20/1678 [00:27<38:18,  1.39s/it]

{'loss': 3.2787, 'learning_rate': 4.9404052443384986e-05, 'epoch': 0.01}


  2%|▏         | 30/1678 [00:42<42:26,  1.55s/it]

...

100%|█████████▉| 1670/1678 [40:32<00:11,  1.43s/it]

{'loss': 2.2839, 'learning_rate': 2.3837902264600714e-07, 'epoch': 0.99}


100%|██████████| 1678/1678 [40:44<00:00,  1.46s/it]

{'train_runtime': 2444.6557, 'train_samples_per_second': 10.986, 'train_steps_per_second': 0.686, 'train_loss': 2.6338295271626815, 'epoch': 1.0}


TrainOutput(global_step=1678, training_loss=2.6338295271626815, metrics={'train_runtime': 2444.6557, 'train_samples_per_second': 10.986, 'train_steps_per_second': 0.686, 'train_loss': 2.6338295271626815, 'epoch': 1.0})

## Step8 模型推理

In [29]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [32]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
res = pipe(ipt, max_length=256, do_sample=True, num_return_sequences=4)
print(res)

[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试有很多技巧。这里列举一些常见的方法:\n\n1. 提前准备：使用备考资料中的提醒来准备考试。例如，列出过去几年的考试时间和题源，以便在考试前做好准备。\n\n2. 保持良好的学习习惯：坚持每天学习，保持充足的睡眠质量和良好的生活环境，并且不断更新自己和学习资料，以备考更加顺利。\n\n3. 复习巩固：根据考试科目和考试主题，通过不断练习来巩固所学知识，并确保充分掌握所要技能。\n\n4. 勤奋努力：多参加各类考试，以便在考试中取得优异的成绩。这样，可以提高你的学习效率和考试能力。\n\n总之，坚持上述方法，不仅能够帮助你应对考试的氛围，还会帮助你在学习和生活中取得更好的成绩。\n\n再次提示，无论是在线教学还是传统课堂教学，都要根据自己需要来学习和备考知识。只要保持学习态度和知识技能，就能取得优异的成绩。'}, {'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧有很多种，每种技巧都会涉及不同的类型。例如，题目技巧通常会使用一些结构化的步骤和语法组织来帮助考生更好地思考题。解题技巧也常常用于快速解决题目，帮助学生更好地掌握考试内容。除此之外，复习技巧也会影响到考试的成功，帮助学生取得更好的成绩。总之，考试技巧对于考试的成功至关重要。'}, {'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧包括多种方面，其中最常见的技巧有：考试时间限制，准备考试材料，认真复习，以及应对考试的策略。考试技巧包括书写技巧，逻辑思考技巧，考试技巧和经验。此外，每个人都可以学习和改进考试技巧，以保持学习的有效性和有效性。'}, {'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧有很多，这里有一些常见的技巧：\n\n1. 组织：考前制定计划并按计划执行。将目标组织好，并在每道题目完成后按时完成。\n2. 掌握公式：一题一答案。可以按照逻辑顺序整理答案。\n3. 思考问题：设题时，要注意仔细思考每一个题目，这样可以避免因遗漏或拼写错误而犯错时。\n4. 控制时间: 在考试开始之前，要认真准备练习，避免在考试时不掌握题目要求

In [33]:
for item in res:
    print('+++'*20)
    print(item)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试有很多技巧。这里列举一些常见的方法:\n\n1. 提前准备：使用备考资料中的提醒来准备考试。例如，列出过去几年的考试时间和题源，以便在考试前做好准备。\n\n2. 保持良好的学习习惯：坚持每天学习，保持充足的睡眠质量和良好的生活环境，并且不断更新自己和学习资料，以备考更加顺利。\n\n3. 复习巩固：根据考试科目和考试主题，通过不断练习来巩固所学知识，并确保充分掌握所要技能。\n\n4. 勤奋努力：多参加各类考试，以便在考试中取得优异的成绩。这样，可以提高你的学习效率和考试能力。\n\n总之，坚持上述方法，不仅能够帮助你应对考试的氛围，还会帮助你在学习和生活中取得更好的成绩。\n\n再次提示，无论是在线教学还是传统课堂教学，都要根据自己需要来学习和备考知识。只要保持学习态度和知识技能，就能取得优异的成绩。'}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧有很多种，每种技巧都会涉及不同的类型。例如，题目技巧通常会使用一些结构化的步骤和语法组织来帮助考生更好地思考题。解题技巧也常常用于快速解决题目，帮助学生更好地掌握考试内容。除此之外，复习技巧也会影响到考试的成功，帮助学生取得更好的成绩。总之，考试技巧对于考试的成功至关重要。'}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧包括多种方面，其中最常见的技巧有：考试时间限制，准备考试材料，认真复习，以及应对考试的策略。考试技巧包括书写技巧，逻辑思考技巧，考试技巧和经验。此外，每个人都可以学习和改进考试技巧，以保持学习的有效性和有效性。'}
+++++++++++++++++++++++++++++++++++++++++++++++++